In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [3]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
public748 = pd.read_csv('input/public748.csv')
public716 = pd.read_csv('input/public716.csv')

In [12]:
model_paths = [
    Path('results/aug_04'),
    Path('results/ds_05'),
    Path('results/ds_05_aug2'),
    Path('results/ds_06/'),
    Path('results/ds_07/'),
    Path('results/ds_08/'),
    Path('results/model_00/'),
    Path('results/ds_09/'),
    Path('results/ds_12/'),
    Path('results/res_00/'),
]


In [13]:
prediction_df = {}
for p in model_paths:
    if (p/'predictions_tta_wo_anomaly.npy').exists():
        prediction_df[p.stem] = sigmoid(np.load(p/'predictions_tta_wo_anomaly.npy')).reshape(-1)
    else:
        pred = sigmoid(np.load(p/'predictions_tta.npy'))
        print(pred.shape)
        if pred.shape[1] == 1:
            pred = pred.reshape(-1)
        elif pred.shape[0] == 5:
            pred = pred[0].reshape(-1)
        prediction_df[p.stem] = pred
prediction_df = pd.DataFrame(prediction_df)
prediction_df['public748'] = public748['target'].values
prediction_df['public716'] = public716['target'].values

(5, 7975, 1)
(5, 7975, 1)


In [14]:
prediction_df

,aug_04,ds_05,ds_05_aug2,ds_06,ds_07,ds_08,model_00,ds_09,ds_12,res_00,public748,public716
0,0.480872,0.336692,0.509052,0.461881,0.457683,0.654743,0.401980,0.529081,0.408687,0.203641,0.077132,0.657053
1,0.994986,0.416911,0.861430,0.788342,0.960843,0.887126,0.882469,0.973470,0.935896,0.624078,0.848641,0.768253
2,0.399984,0.336593,0.398234,0.448686,0.398339,0.425958,0.398277,0.354265,0.425394,0.196160,0.106932,0.473940
3,0.382961,0.332337,0.438421,0.442875,0.395457,0.440116,0.452815,0.479609,0.404383,0.212767,0.256063,0.398179
4,0.413334,0.333511,0.434260,0.489413,0.550811,0.493679,0.390727,0.614418,0.554948,0.259297,0.507310,0.489631
...,...,...,...,...,...,...,...,...,...,...,...,...
7970,0.996306,0.496162,0.971251,0.990373,0.986535,0.994714,0.648365,0.999868,0.966139,0.300147,0.937471,0.865461
7971,0.340874,0.333426,0.386766,0.446710,0.439642,0.447186,0.375045,0.428340,0.480493,0.248835,0.088365,0.585368
7972,0.335046,0.330965,0.385837,0.397676,0.349482,0.416949,0.373027,0.306392,0.349015,0.204874,0.081671,0.452188
7973,0.447237,0.344412,0.418369,0.502658,0.411904,0.474843,0.447379,0.360838,0.432723,0.207938,0.112314,0.571307


In [18]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
test['target'] = prediction_df['ds_09'] * 0.5 + prediction_df['public748'] * 0.5
# test['target'] = prediction_df['res_00']

In [19]:
test

,id,target
0,00054c878,0.303107
1,0007285a3,0.911056
2,00076c5a6,0.230599
3,001349290,0.367836
4,001a52e92,0.560864
...,...,...
7970,ffbce04ef,0.968669
7971,ffc2d976b,0.258352
7972,ffc905909,0.194032
7973,ffe276f3e,0.236576


In [20]:
test.to_csv('results/blend_submission.csv', index=False)
# test.to_csv(Path('results/res_00/')/'submission.csv', index=False)